# Pre processing UCI-HAR

This notebook will pre processing the dataset UCI-HAR (https://archive.ics.uci.edu/ml/datasets/human+activity+recognition+using+smartphones)

1. Load the uci dataset
2. Remove the overlap from samples
3. Remove the gravity appling a ButterWoeth filter
4. Resample the data from 50Hz to 20Hz
5. Change the time window from 2.56s to 3s
6. Change the accelerometer mensure from g to m/s²

In [1]:
import numpy as np
import pandas as pd
import os
import plotly.express as px
import plotly.graph_objects as go

## Filtering the accelerometer signal

In [2]:
from scipy import signal

def filtering(sig):
    
    sos = signal.butter(3, 0.3, 'low', fs=50, output='sos')
    filtered = signal.sosfilt(sos, sig)
    return filtered

def resample(sig, time_window):
    return signal.resample(sig, 20*time_window)

In [3]:
def transformar(X, y, user, sensor, filter_the_signal=True):
    """Essa função é responsável por pegar os dados do UCI-HAR
    (recortados em janelas de 2,56 segundos e com sobreposição de 50%)
    e transformá-los em janelas de 3 segundos sem sobreposição"""
    '''
    '''
    
    # Vamos ver quando ocorre uma nova amostra (ou seja, quando não ocorre sobreposição)
    aux = np.zeros(len(y)-1)
    for i in range(len(y)-1):
        aux[i] = np.all(X[i,64:]!=X[i+1,:64])
    ind = np.nonzero(aux)[0]+1
    ind = np.append([0],ind)
    ind = np.append(ind,[len(y)])
    
    # Agora vamos construir as capturas originais (sem sobreposição)
    sigs = []
    yy = []
    uuser = []
    for i in range(len(ind)-1):
        sig = X[ind[i]:ind[i+1],64:].reshape((ind[i+1]-ind[i])*64)
        sig = np.append(X[ind[i],:64],sig)
        
        if sensor < 3:
            if filter_the_signal:
                sig_filtered = sig - filtering(sig) # Removing the gravity
            else:
                sig_filtered = sig
            time_window = len(sig_filtered) // 50 # The time window is the time of the total window
            sig_resampled = resample(sig_filtered, time_window) # Resampling the signal

            sigs.append(sig_resampled)
            yy.append(y[ind[i]])
            uuser.append(user[ind[i]])
        else:
            time_window = len(sig) // 50 # The time window is the time of the total window
            sig_resampled = resample(sig, time_window) # Resampling the signal

            sigs.append(sig_resampled)
            yy.append(y[ind[i]])
            uuser.append(user[ind[i]])
    # Vamos construir as janelas de 3 segundos (lembrando que o UCI-HAR está amostrado a uma taxa de 20 Hz)
    new_time_window = 3
    sample_rate = 20
    points_per_new_window = new_time_window*sample_rate
    
    aux = np.zeros(len(sigs))
    for i in range(len(sigs)):
        aux[i] = len(sigs[i])
    Na = np.sum(aux//points_per_new_window, dtype=np.int64)
    mat = np.zeros((Na,points_per_new_window))
    yyy = np.zeros(Na)
    uuuser = np.zeros(Na)
    k = 0
    for i in range(len(sigs)):
        for j in range(len(sigs[i])//points_per_new_window):
            mat[k,:] = sigs[i][j*points_per_new_window:(j+1)*points_per_new_window]
            yyy[k] = yy[i]
            uuuser[k] = uuser[i]
            k+=1
    return mat, yyy, uuuser;

## Pre processing the train data

In [4]:
# Vamos abrir os dados de treino do UCI-HAR
pasta = "../../../../../Downloads/UCI HAR Dataset/UCI HAR Dataset/train/"
# dados = [None]*6
sensor = [
    'Inertial Signals/total_acc_x_train.txt', 
    'Inertial Signals/total_acc_y_train.txt', 
    'Inertial Signals/total_acc_z_train.txt', 
    'Inertial Signals/body_gyro_x_train.txt', 
    'Inertial Signals/body_gyro_y_train.txt', 
    'Inertial Signals/body_gyro_z_train.txt'
]
dados = [np.loadtxt(pasta+sensor[i]) for i in range(6)]
                    
# dados[0] = np.loadtxt(pasta+'Inertial Signals/total_acc_x_train.txt')
# dados[1] = np.loadtxt(pasta+'Inertial Signals/total_acc_y_train.txt')
# dados[2] = np.loadtxt(pasta+'Inertial Signals/total_acc_z_train.txt')
# dados[3] = np.loadtxt(pasta+'Inertial Signals/body_gyro_x_train.txt')
# dados[4] = np.loadtxt(pasta+'Inertial Signals/body_gyro_y_train.txt')
# dados[5] = np.loadtxt(pasta+'Inertial Signals/body_gyro_z_train.txt')
                    
y = np.loadtxt(pasta+'y_train.txt')
user = np.loadtxt(pasta+'subject_train.txt')
labels = ['WALKING', 'WALKING_UPSTAIRS', 'WALKING_DOWNSTAIRS', 'SITTING', 'STANDING', 'LAYING']
print('Número de amostras por classe \npara janelas de 2,56 segundos e sobreposição de 50%')
for i in np.unique(y):
    print(labels[i.astype(int)-1]+': '+str(np.sum(y==i)))

Número de amostras por classe 
para janelas de 2,56 segundos e sobreposição de 50%
WALKING: 1226
WALKING_UPSTAIRS: 1073
WALKING_DOWNSTAIRS: 986
SITTING: 1286
STANDING: 1374
LAYING: 1407


In [5]:
new_time_window = 3
sample_rate = 20
points_per_new_window = new_time_window*sample_rate

colunas = ['accel-x-' + str(x) for x in np.arange(points_per_new_window).tolist()]
colunas += ['accel-y-' + str(x) for x in np.arange(points_per_new_window).tolist()]
colunas += ['accel-z-' + str(x) for x in np.arange(points_per_new_window).tolist()]
colunas += ['gyro-x-' + str(x) for x in np.arange(points_per_new_window).tolist()]
colunas += ['gyro-y-' + str(x) for x in np.arange(points_per_new_window).tolist()]
colunas += ['gyro-z-' + str(x) for x in np.arange(points_per_new_window).tolist()]
colunas += ['activity code']
colunas += ['user']

In [6]:
new_time_window = 3
sample_rate = 20
points_per_new_window = new_time_window*sample_rate

train = []
for i in range(6):
    mat, yyy, uuuser = transformar(dados[i], y, user, i)
    if type(train) == list:
        train = mat
    else:
        train = np.concatenate([train, mat], axis=1)
    print(train.shape)
train = np.column_stack((train,yyy.astype(int),uuuser.astype(int)))

(3049, 60)
(3049, 120)
(3049, 180)
(3049, 240)
(3049, 300)
(3049, 360)


In [7]:
df_train = pd.DataFrame(data=train)

In [8]:
df_train.columns = colunas
df_train = df_train.astype({"activity code": int, "user": int})

# Removing the activity laying
df_train = df_train[df_train['activity code'].isin([1, 2, 3, 4, 5])]
df_train

,accel-x-0,accel-x-1,accel-x-2,accel-x-3,accel-x-4,accel-x-5,accel-x-6,accel-x-7,accel-x-8,accel-x-9,...,gyro-z-52,gyro-z-53,gyro-z-54,gyro-z-55,gyro-z-56,gyro-z-57,gyro-z-58,gyro-z-59,activity code,user
0,0.709658,1.098709,0.974849,1.042785,0.988832,1.020784,0.979064,0.994043,0.956313,0.957692,...,0.001605,-0.000460,-0.001029,-0.000238,0.002616,0.006758,0.009952,-0.005709,5,1
1,-0.062805,-0.056817,-0.051473,-0.048221,-0.045523,-0.037635,-0.031380,-0.030902,-0.026130,-0.022408,...,-0.001788,-0.005908,-0.010234,-0.011284,-0.009106,-0.001114,-0.001296,-0.002068,5,1
2,-0.002409,-0.004017,-0.003232,-0.001114,-0.001560,-0.004402,-0.002449,-0.003568,-0.001499,-0.002786,...,0.000421,0.004503,0.002473,0.001083,0.007256,0.021702,0.007641,0.009008,5,1
3,-0.003230,0.000680,0.000479,0.000186,0.004959,-0.002651,0.000215,-0.001799,0.002436,0.001305,...,0.005918,-0.003793,-0.002657,0.001718,-0.007073,-0.012203,-0.009020,-0.008270,5,1
4,0.002240,0.000876,-0.002080,0.001666,0.000811,-0.000224,-0.001148,-0.002598,0.005442,-0.000696,...,0.000288,-0.001961,0.001915,0.002949,0.007039,-0.004036,-0.003410,-0.004388,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,-0.040509,0.051099,0.043308,0.131483,0.405293,0.583331,0.315966,0.023301,-0.052495,-0.145659,...,-0.396459,-0.545852,-0.702665,-0.833772,-0.654115,-0.355501,-0.202928,-0.085239,2,30
3045,0.714841,1.279840,1.410648,1.509643,1.230629,0.989366,0.834677,0.837818,0.574001,0.602336,...,-0.445656,-0.579521,-0.846560,-0.700226,-0.574337,-0.199706,0.069693,0.210308,2,30
3046,0.201737,0.463930,0.421784,0.134871,-0.079729,-0.105013,-0.213468,-0.364409,-0.369793,-0.380467,...,0.035472,0.292552,0.203059,-0.175440,0.281563,0.134684,0.217240,0.443779,2,30
3047,-0.249057,-0.391390,-0.204677,-0.254794,-0.239145,-0.136961,0.073090,0.282398,0.715923,0.157419,...,-0.232415,0.024815,0.065954,0.068590,0.115724,0.091088,0.309496,0.256306,2,30


In [9]:
print('Número de amostras por classe na formatação final:')
for i in np.unique(yyy):
    print(labels[i.astype(int)-1]+': '+str(np.sum(yyy==i)))

Número de amostras por classe na formatação final:
WALKING: 506
WALKING_UPSTAIRS: 439
WALKING_DOWNSTAIRS: 395
SITTING: 544
STANDING: 575
LAYING: 590


In [10]:
df_train.iloc[:,:181]

,accel-x-0,accel-x-1,accel-x-2,accel-x-3,accel-x-4,accel-x-5,accel-x-6,accel-x-7,accel-x-8,accel-x-9,...,accel-z-51,accel-z-52,accel-z-53,accel-z-54,accel-z-55,accel-z-56,accel-z-57,accel-z-58,accel-z-59,gyro-x-0
0,0.709658,1.098709,0.974849,1.042785,0.988832,1.020784,0.979064,0.994043,0.956313,0.957692,...,-0.015468,-0.009330,-0.023820,-0.025836,-0.015847,-0.017333,-0.010779,-0.011047,-0.009448,0.024057
1,-0.062805,-0.056817,-0.051473,-0.048221,-0.045523,-0.037635,-0.031380,-0.030902,-0.026130,-0.022408,...,-0.008138,-0.002905,-0.005300,-0.006094,-0.002435,0.001763,-0.002770,-0.009235,-0.005878,0.012282
2,-0.002409,-0.004017,-0.003232,-0.001114,-0.001560,-0.004402,-0.002449,-0.003568,-0.001499,-0.002786,...,0.001453,-0.000702,0.002514,0.002652,0.001375,-0.000075,0.003720,-0.007147,0.011641,-0.007669
3,-0.003230,0.000680,0.000479,0.000186,0.004959,-0.002651,0.000215,-0.001799,0.002436,0.001305,...,0.001023,-0.004886,-0.003029,0.003908,0.006433,0.005771,-0.000347,0.005410,-0.002820,-0.003489
4,0.002240,0.000876,-0.002080,0.001666,0.000811,-0.000224,-0.001148,-0.002598,0.005442,-0.000696,...,0.004948,0.000535,0.006637,0.004178,0.002398,0.010023,-0.004089,-0.001672,0.009354,0.009188
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,-0.040509,0.051099,0.043308,0.131483,0.405293,0.583331,0.315966,0.023301,-0.052495,-0.145659,...,0.158205,-0.001717,-0.036479,-0.102683,-0.048426,-0.147524,-0.182708,-0.187176,-0.162139,0.322930
3045,0.714841,1.279840,1.410648,1.509643,1.230629,0.989366,0.834677,0.837818,0.574001,0.602336,...,-0.004219,-0.093370,-0.095662,-0.079062,-0.151032,-0.179563,-0.148950,-0.137044,-0.177787,-0.493072
3046,0.201737,0.463930,0.421784,0.134871,-0.079729,-0.105013,-0.213468,-0.364409,-0.369793,-0.380467,...,-0.113754,-0.129466,-0.112359,-0.214151,-0.396940,-0.241146,-0.220447,-0.165437,-0.133502,-0.517789
3047,-0.249057,-0.391390,-0.204677,-0.254794,-0.239145,-0.136961,0.073090,0.282398,0.715923,0.157419,...,-0.156416,-0.075032,-0.039497,-0.086440,-0.060004,0.031740,0.140555,0.201651,0.328831,0.519843


In [11]:
# Mudando a unidade de medida do acelerômetro para m/s² (Basta multiplicar por g = 9.81)
# g = 9.81
g = 9.81
df_train.iloc[:,:180] = df_train.iloc[:,:180]*g
df_train

,accel-x-0,accel-x-1,accel-x-2,accel-x-3,accel-x-4,accel-x-5,accel-x-6,accel-x-7,accel-x-8,accel-x-9,...,gyro-z-52,gyro-z-53,gyro-z-54,gyro-z-55,gyro-z-56,gyro-z-57,gyro-z-58,gyro-z-59,activity code,user
0,6.961742,10.778332,9.563269,10.229716,9.700441,10.013894,9.604622,9.751559,9.381431,9.394958,...,0.001605,-0.000460,-0.001029,-0.000238,0.002616,0.006758,0.009952,-0.005709,5,1
1,-0.616115,-0.557374,-0.504948,-0.473043,-0.446585,-0.369201,-0.307839,-0.303152,-0.256335,-0.219824,...,-0.001788,-0.005908,-0.010234,-0.011284,-0.009106,-0.001114,-0.001296,-0.002068,5,1
2,-0.023630,-0.039409,-0.031710,-0.010933,-0.015305,-0.043188,-0.024020,-0.035000,-0.014708,-0.027331,...,0.000421,0.004503,0.002473,0.001083,0.007256,0.021702,0.007641,0.009008,5,1
3,-0.031687,0.006668,0.004699,0.001821,0.048644,-0.026002,0.002109,-0.017645,0.023892,0.012797,...,0.005918,-0.003793,-0.002657,0.001718,-0.007073,-0.012203,-0.009020,-0.008270,5,1
4,0.021979,0.008597,-0.020402,0.016348,0.007953,-0.002194,-0.011266,-0.025485,0.053391,-0.006831,...,0.000288,-0.001961,0.001915,0.002949,0.007039,-0.004036,-0.003410,-0.004388,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,-0.397397,0.501285,0.424851,1.289849,3.975923,5.722475,3.099624,0.228584,-0.514974,-1.428918,...,-0.396459,-0.545852,-0.702665,-0.833772,-0.654115,-0.355501,-0.202928,-0.085239,2,30
3045,7.012588,12.555226,13.838454,14.809593,12.072467,9.705683,8.188178,8.218991,5.630945,5.908920,...,-0.445656,-0.579521,-0.846560,-0.700226,-0.574337,-0.199706,0.069693,0.210308,2,30
3046,1.979043,4.551149,4.137700,1.323084,-0.782141,-1.030182,-2.094125,-3.574853,-3.627672,-3.732386,...,0.035472,0.292552,0.203059,-0.175440,0.281563,0.134684,0.217240,0.443779,2,30
3047,-2.443248,-3.839537,-2.007886,-2.499533,-2.346011,-1.343588,0.717010,2.770320,7.023206,1.544278,...,-0.232415,0.024815,0.065954,0.068590,0.115724,0.091088,0.309496,0.256306,2,30


## Pre processing the test data

In [12]:
# Agora vamos abrir os dados de treino do UCI-HAR
pasta = "../../../../../Downloads/UCI HAR Dataset/UCI HAR Dataset/test/"
dados = [None]*6
sensor = [
    'Inertial Signals/total_acc_x_test.txt', 
    'Inertial Signals/total_acc_y_test.txt', 
    'Inertial Signals/total_acc_z_test.txt', 
    'Inertial Signals/body_gyro_x_test.txt', 
    'Inertial Signals/body_gyro_y_test.txt', 
    'Inertial Signals/body_gyro_z_test.txt'
]
dados = [np.loadtxt(pasta+sensor[i]) for i in range(6)]
                    
y = np.loadtxt(pasta+'y_test.txt')
user = np.loadtxt(pasta+'subject_test.txt')
labels = ['WALKING', 'WALKING_UPSTAIRS', 'WALKING_DOWNSTAIRS', 'SITTING', 'STANDING'
          , 'LAYING']
print('Número de amostras por classe \npara janelas de 2,56 segundos e sobreposição de 50%')
for i in np.unique(y):
    print(labels[i.astype(int)-1]+': '+str(np.sum(y==i)))

Número de amostras por classe 
para janelas de 2,56 segundos e sobreposição de 50%
WALKING: 496
WALKING_UPSTAIRS: 471
WALKING_DOWNSTAIRS: 420
SITTING: 491
STANDING: 532
LAYING: 537


In [13]:
new_time_window = 3
sample_rate = 20
points_per_new_window = new_time_window*sample_rate

test = []
for i in range(6):
    mat, yyy, uuuser = transformar(dados[i], y, user, i)
    if type(test) == list:
        test = mat
    else:
        test = np.concatenate([test, mat], axis=1)
    print(test.shape)
test = np.column_stack((test,yyy.astype(int),uuuser.astype(int)))

(1224, 60)
(1224, 120)
(1224, 180)
(1224, 240)
(1224, 300)
(1224, 360)


In [14]:
df_test = pd.DataFrame(data=test)

In [15]:
df_test.columns = colunas
df_test = df_test.astype({"activity code": int, "user": int})

# Removing the activity laying
df_test = df_test[df_test['activity code'].isin([1, 2, 3, 4, 5])]
df_test

,accel-x-0,accel-x-1,accel-x-2,accel-x-3,accel-x-4,accel-x-5,accel-x-6,accel-x-7,accel-x-8,accel-x-9,...,gyro-z-52,gyro-z-53,gyro-z-54,gyro-z-55,gyro-z-56,gyro-z-57,gyro-z-58,gyro-z-59,activity code,user
0,0.710759,1.132157,0.945674,0.961547,0.966303,1.003419,0.959539,0.965556,0.918833,0.938538,...,0.020261,0.019462,0.031394,0.031187,0.028060,0.034830,0.026174,0.013201,5,2
1,-0.054172,-0.047570,-0.041633,-0.033630,-0.037490,-0.031935,-0.030124,-0.016172,-0.017133,-0.015793,...,-0.010148,-0.007775,-0.005448,-0.002262,-0.005768,-0.007097,-0.003873,-0.005801,5,2
2,-0.000734,-0.001002,-0.004313,-0.001778,-0.005900,-0.006174,-0.000228,0.004919,0.001379,-0.002907,...,-0.031461,-0.035463,-0.008491,-0.002806,0.007795,0.011203,0.003488,-0.004301,5,2
3,0.000279,0.001072,0.000962,0.001843,0.002082,0.004419,0.005007,0.001154,0.002870,-0.001084,...,-0.010083,-0.005960,-0.006940,-0.009337,-0.009792,-0.001859,-0.004210,-0.000520,5,2
4,0.000210,-0.002119,-0.004740,-0.002119,-0.001793,-0.002206,-0.000874,0.001632,0.001275,-0.003247,...,0.003836,0.001970,0.003919,0.005948,-0.002991,0.006746,0.007876,0.006598,5,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1219,-0.263123,-0.368496,-0.210964,-0.092818,0.021278,0.256880,0.620826,0.423931,0.060889,-0.053062,...,0.117826,-0.216337,-0.498443,-0.221196,0.040862,0.122537,0.130663,0.001037,3,24
1220,0.669263,1.331842,1.227849,1.228038,1.342306,0.990068,0.768119,0.892655,0.853070,0.756679,...,-0.128649,0.097912,0.551742,0.356796,-0.128970,-0.070716,0.436967,-0.035641,2,24
1221,0.011700,-0.093523,-0.211989,-0.071201,-0.217767,-0.275336,-0.237745,-0.151789,-0.055965,0.045442,...,-0.147423,-0.101344,-0.070548,0.196283,0.036365,-0.383925,-0.499610,-0.496780,2,24
1222,-0.002811,-0.148740,-0.323136,-0.256827,-0.092179,-0.095522,0.068596,0.129345,0.168141,0.411807,...,-0.345504,-0.396574,-0.585145,-0.923041,-0.865991,-0.622471,-0.736700,-0.851594,2,24


In [16]:
print('Número de amostras por classe na formatação final:')
for i in np.unique(yyy):
    print(labels[i.astype(int)-1]+': '+str(np.sum(yyy==i)))

Número de amostras por classe na formatação final:
WALKING: 204
WALKING_UPSTAIRS: 189
WALKING_DOWNSTAIRS: 173
SITTING: 204
STANDING: 227
LAYING: 227


In [17]:
# Mudando a unidade de medida do acelerômetro para m/s² (Basta multiplicar por g = 9.81)
# g = 9.81
g = 9.81
df_test.iloc[:,:180] = df_test.iloc[:,:180]*g
df_test

,accel-x-0,accel-x-1,accel-x-2,accel-x-3,accel-x-4,accel-x-5,accel-x-6,accel-x-7,accel-x-8,accel-x-9,...,gyro-z-52,gyro-z-53,gyro-z-54,gyro-z-55,gyro-z-56,gyro-z-57,gyro-z-58,gyro-z-59,activity code,user
0,6.972541,11.106462,9.277066,9.432781,9.479432,9.843543,9.413076,9.472105,9.013756,9.207058,...,0.020261,0.019462,0.031394,0.031187,0.028060,0.034830,0.026174,0.013201,5,2
1,-0.531431,-0.466659,-0.408417,-0.329905,-0.367775,-0.313287,-0.295520,-0.158646,-0.168076,-0.154934,...,-0.010148,-0.007775,-0.005448,-0.002262,-0.005768,-0.007097,-0.003873,-0.005801,5,2
2,-0.007196,-0.009831,-0.042308,-0.017440,-0.057883,-0.060567,-0.002234,0.048251,0.013527,-0.028520,...,-0.031461,-0.035463,-0.008491,-0.002806,0.007795,0.011203,0.003488,-0.004301,5,2
3,0.002739,0.010520,0.009438,0.018075,0.020421,0.043346,0.049119,0.011324,0.028157,-0.010630,...,-0.010083,-0.005960,-0.006940,-0.009337,-0.009792,-0.001859,-0.004210,-0.000520,5,2
4,0.002062,-0.020784,-0.046501,-0.020784,-0.017592,-0.021642,-0.008572,0.016014,0.012508,-0.031853,...,0.003836,0.001970,0.003919,0.005948,-0.002991,0.006746,0.007876,0.006598,5,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1219,-2.581241,-3.614946,-2.069556,-0.910543,0.208734,2.519993,6.090305,4.158760,0.597321,-0.520535,...,0.117826,-0.216337,-0.498443,-0.221196,0.040862,0.122537,0.130663,0.001037,3,24
1220,6.565466,13.065374,12.045194,12.047053,13.168018,9.712563,7.535252,8.756949,8.368613,7.423018,...,-0.128649,0.097912,0.551742,0.356796,-0.128970,-0.070716,0.436967,-0.035641,2,24
1221,0.114777,-0.917459,-2.079612,-0.698483,-2.136296,-2.701049,-2.332279,-1.489051,-0.549015,0.445787,...,-0.147423,-0.101344,-0.070548,0.196283,0.036365,-0.383925,-0.499610,-0.496780,2,24
1222,-0.027578,-1.459136,-3.169965,-2.519476,-0.904274,-0.937075,0.672922,1.268873,1.649460,4.039828,...,-0.345504,-0.396574,-0.585145,-0.923041,-0.865991,-0.622471,-0.736700,-0.851594,2,24


In [18]:
df_train.to_csv('../../data_2/views/UCI-HAR/unbalanced_view_filtered_acc_9.81_train_test-resampled_20hz-v1/train.csv',index=False)
df_test.to_csv('../../data_2/views/UCI-HAR/unbalanced_view_filtered_acc_9.81_train_test-resampled_20hz-v1/test.csv',index=False)